In [7]:
import numpy as np
import pandas as pd
from datetime import datetime
from pytimeparse2 import parse, disable_dateutil
disable_dateutil()

In [8]:
attention_check_index = [
    '15', '26', '37', '48', '59', '70', '81', '92', '103', '114'
]

In [9]:
U01_time = pd.DataFrame(columns=["start_time", "end_time"])
U01_time[["start_time", "end_time"]] = pd.read_csv('time/log_time/U01.csv').T

study_start_time = datetime.strptime("2023-08-27 10:01:27", "%Y-%m-%d %H:%M:%S")
U01_time["start_time"] = U01_time["start_time"].apply(lambda x: study_start_time + parse(x.strip().replace(" ", ":"), as_timedelta=True))
U01_time["end_time"] = U01_time["end_time"].apply(lambda x: study_start_time + parse(x.strip().replace(" ", ":"), as_timedelta=True))
U01_time["duration_in_seconds"] = U01_time["end_time"] - U01_time["start_time"]
U01_time["duration_in_seconds"] = U01_time["duration_in_seconds"].apply(lambda x: x.total_seconds())
U01_time.drop(index=attention_check_index, inplace=True)
U01_time.reset_index(drop=True, inplace=True)

U01_time.to_csv("time/log_time/U01_time.csv", index=False)

with open("log/U01.log", "w") as f:
    f.write(str(len(U01_time)))
    f.write("\n")
    f.write("0")
    f.write("\n")

In [10]:
def read_log(path):
    with open(path, 'r') as f:
        lines = f.readlines()
    return list(map(lambda x: int(x.strip()), lines[-2:]))

In [11]:
no_pref_df = pd.DataFrame(columns=['uid', 'num_no_pref', 'total_num'])
for i in range(1, 43):
    uid = f"U{i:02d}"
    path = f"./log/{uid}.log"
    total, num_no_pref = read_log(path)
    no_pref_df = pd.concat(
        [no_pref_df, pd.DataFrame([[uid, num_no_pref, total]], columns=['uid', 'num_no_pref', 'total_num'])],
        axis=0,
        ignore_index=True
    )

In [12]:
no_pref_df[["uid", "num_no_pref"]].to_csv("time/num_no_pref.csv", index=False)